# T1 - Whirlwind tour

Installing and getting started with HPVsim is quite simple.

HPVsim is a Python package that can be pip-installed by typing `pip install hpvsim` into a terminal. You can then check that the installation was successful by importing HPVsim with `import hpvsim as hpv`.

The basic design philosophy of HPVsim is: **common tasks should be simple**. For example:

- Defining parameters
- Running a simulation
- Plotting results

This tutorial walks you through how to do these things.

<div class="alert alert-info">
    
Click [here](https://mybinder.org/v2/gh/institutefordiseasemodeling/hpvsim/HEAD?urlpath=lab%2Ftree%2Fdocs%2Ftutorials%2Ftut_intro.ipynb) to open an interactive version of this notebook.
    
</div>

## Hello world

To create, run, and plot a sim with default options is just:

In [1]:
import sciris as sc

sc.findinds([1,2,3],2)

array([1])